In [1]:
USE [CRM_203801]

Commands completed successfully.

Total execution time: 00:00:00.170

In [2]:
SET ANSI_NULLS OFF
GO

SET QUOTED_IDENTIFIER ON
GO

CREATE TABLE [dbo].[TbDiario_Auditoria](
	[IdDiarioAuditoria] [bigint] IDENTITY(1,1) NOT NULL,
	[DataHora] [datetime] NOT NULL,
	[Operacao] [char](1) NOT NULL,
	[IdUsuario] [int] NULL,

	[IdDiario] [int] NULL,
	[IdTurma] [int] NULL,
	[IdDisciplina] [int] NULL,
	[Descricao] [varchar](50) NULL,
	[IdFaseNota] [int] NULL,
	[TipoConfirmacao] [char](1) NULL,
	[QtdeMaximaAulas] [int] NULL,
	[DataLimiteDigitacao] [datetime] NULL,
	[QtdeMinimaAulas] [int] NULL,
	[StRegistroAulaPeriodoFaseNota] [bit] NULL,
	[DataBloqueioDigitacaoAula] [datetime] NULL,
 CONSTRAINT [PK_TbDiario_Auditoria] PRIMARY KEY CLUSTERED 
(
	[IdDiarioAuditoria] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.530

In [4]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER OFF
GO

--
-- Activesoft Consultoria 
-- TRIGGER criada pelo Activesoft Service Pack em 21/01/2020 durante atualização para versão 1.065.xxx.
--

CREATE TRIGGER [dbo].[TgDiario_Auditoria] ON [dbo].[TbDiario]
--ALTER TRIGGER [dbo].[TgDiario_Auditoria] ON [dbo].[TbDiario]
FOR INSERT, UPDATE, DELETE
AS
BEGIN
	SET NOCOUNT ON

	IF ( SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS c WHERE DATA_TYPE NOT IN ( 'text', 'image' ) AND TABLE_NAME = 'TbDiario' AND LEFT(COLUMN_NAME,1) <> '_' ) <> 11
	BEGIN
		ROLLBACK TRANSACTION 
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgDiario_Auditoria inválida)', 16, 1) 
		RETURN 
	END 

	
	DECLARE @IdUsuario INTEGER
	IF APP_NAME() = 'SIGAWEB'
		SELECT	@IdUsuario = IdUsuario
		FROM	dbo.FnCriarTabelaTemporaria(HOST_NAME())		
	ELSE IF OBJECT_ID( 'tempdb..#TempTable' ) IS NOT NULL
		SELECT @IdUsuario = IdUsuario FROM #TempTable

	IF ( @IdUsuario IS NULL ) 
	BEGIN
		ROLLBACK TRANSACTION 
		RAISERROR ('A auditoria não está funcionando corretamente. Reinicie o sistema. (TgDiario_Auditoria)', 16, 1)
		RETURN 
	END

	
	DECLARE @TipoOperacao CHAR(1) = 'I' -- Insert by default.
	IF EXISTS(SELECT * FROM DELETED)
	BEGIN
	SET @TipoOperacao = 
	    CASE
		WHEN EXISTS(SELECT * FROM INSERTED) THEN 'U' ELSE 'D' 
	    END
	END
	ELSE 
	IF NOT EXISTS(SELECT * FROM INSERTED) RETURN; -- Nem updated nem inserted. Pode ter sido um "failed delete".


	IF ( @TipoOperacao = 'D' )
		INSERT	TbDiario_Auditoria (	DataHora,	Operacao,		IdUsuario,	IdDiario,	IdTurma,	IdDisciplina,	Descricao,	IdFaseNota,	TipoConfirmacao,	QtdeMaximaAulas, DataLimiteDigitacao, QtdeMinimaAulas, StRegistroAulaPeriodoFaseNota, DataBloqueioDigitacaoAula )
		SELECT							GETDATE(),	@TipoOperacao,	@IdUsuario, IdDiario,	IdTurma,	IdDisciplina,	Descricao,	IdFaseNota,	TipoConfirmacao,	QtdeMaximaAulas, DataLimiteDigitacao, QtdeMinimaAulas, StRegistroAulaPeriodoFaseNota, DataBloqueioDigitacaoAula	
		FROM	deleted
	ELSE
		INSERT	TbDiario_Auditoria (	DataHora,	Operacao,		IdUsuario,	IdDiario,	IdTurma,	IdDisciplina,	Descricao,	IdFaseNota,	TipoConfirmacao,	QtdeMaximaAulas, DataLimiteDigitacao, QtdeMinimaAulas, StRegistroAulaPeriodoFaseNota, DataBloqueioDigitacaoAula )
		SELECT							GETDATE(),	@TipoOperacao,	@IdUsuario, IdDiario,	IdTurma,	IdDisciplina,	Descricao,	IdFaseNota,	TipoConfirmacao,	QtdeMaximaAulas, DataLimiteDigitacao, QtdeMinimaAulas, StRegistroAulaPeriodoFaseNota, DataBloqueioDigitacaoAula	
		FROM	inserted
END


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.697

In [6]:
DROP TRIGGER [dbo].[TgAudit_TbDiarioD]

DROP TRIGGER [dbo].[TgAudit_TbDiarioI]

DROP TRIGGER [dbo].[TgAudit_TbDiarioU]

Commands completed successfully.

Total execution time: 00:00:00.244